In [81]:
import pandas as pd

df=pd.read_csv("../inputs/a2_feat_packet_level_normal_OCT8.csv")
df=df.sort_values(by=['tcp_stream','protocol','start_time'])
print("Total Unidirectional Flows:",len(df))

Total Unidirectional Flows: 269240


In [92]:
df[['flow_start']]=df[['flow_start']].apply(pd.to_datetime)


,tcp_stream,protocol,flow_id,flow_start,flow_finish,src_ip,src_port,dst_ip,dst_port,total_duration,...,bpp,total_http_errors,total_failures,flag_ack,flag_fin,flag_psh,flag_rst,flag_syn,first_flag,last_flag
2,0,HTTP,192.168.10.14:50095->13.107.4.50:80,2017-07-03 12:16:42.169,2017-07-03 12:21:57.939,192.168.10.14,50095,13.107.4.50,80,315.769901000000004387402441352605819702148437...,...,366.70790378006864784765639342367649078369140625,0,0,291,0,291,0,0,NaN,NaN
0,0,HTTP,13.107.4.50:80->192.168.10.14:50095,2017-07-03 12:16:44.006,2017-07-03 12:21:58.191,13.107.4.50,80,192.168.10.14,50095,314.185010000000033869582694023847579956054687...,...,1914.47191011235986479732673615217208862304687500,0,0,178,0,178,0,0,TCP Spurious Retransmission,TCP Spurious Retransmission
3,0,HTTP,192.168.10.15:49647->13.107.4.50:80,2017-07-03 12:34:53.358,2017-07-03 12:42:54.830,192.168.10.15,49647,13.107.4.50,80,481.471517999999946368916425853967666625976562...,...,366.66666666666674245789181441068649291992187500,0,0,465,0,465,0,0,NaN,NaN
1,0,HTTP,13.107.4.50:80->192.168.10.15:49647,2017-07-03 12:34:55.253,2017-07-03 12:42:53.495,13.107.4.50,80,192.168.10.15,49647,478.241279999999960637069307267665863037109375...,...,2008.71649484536078489327337592840194702148437500,0,0,194,0,194,0,0,TCP Spurious Retransmission,TCP Spurious Retransmission
4,0,SSL,23.194.140.85:443->192.168.10.16:49376,2017-07-03 13:18:37.636,2017-07-03 13:18:37.659,23.194.140.85,443,192.168.10.16,49376,0.02327500000000000054956039718945248750969767...,...,1099.66666666666674245789181441068649291992187500,0,0,3,0,2,0,0,TCP Spurious Retransmission,TCP Spurious Retransmission


In [94]:
pd.set_option("display.precision", 50)
df[['flow_start','start_time']].sort_values(by=['flow_start']).head(10)

,flow_start,start_time
18,2017-07-03 11:55:58.598,1499082958.5983080863952636718750
49,2017-07-03 11:56:22.331,1499082982.3310179710388183593750
116610,2017-07-03 11:56:38.027,1499082998.0276010036468505859375
116604,2017-07-03 11:56:38.027,1499082998.0276679992675781250000
116594,2017-07-03 11:56:38.028,1499082998.0285749435424804687500
116593,2017-07-03 11:56:38.028,1499082998.0288438796997070312500
116592,2017-07-03 11:56:38.028,1499082998.0284211635589599609375
116591,2017-07-03 11:56:38.028,1499082998.0284900665283203125000
145050,2017-07-03 11:56:38.029,1499082998.0292079448699951171875
145045,2017-07-03 11:56:38.029,1499082998.0292608737945556640625


In [95]:
df[['flow_start','start_time']].sort_values(by=['start_time']).head(10)

,flow_start,start_time
18,2017-07-03 11:55:58.598,1499082958.5983080863952636718750
49,2017-07-03 11:56:22.331,1499082982.3310179710388183593750
116610,2017-07-03 11:56:38.027,1499082998.0276010036468505859375
116604,2017-07-03 11:56:38.027,1499082998.0276679992675781250000
116592,2017-07-03 11:56:38.028,1499082998.0284211635589599609375
116591,2017-07-03 11:56:38.028,1499082998.0284900665283203125000
116594,2017-07-03 11:56:38.028,1499082998.0285749435424804687500
116593,2017-07-03 11:56:38.028,1499082998.0288438796997070312500
145050,2017-07-03 11:56:38.029,1499082998.0292079448699951171875
145045,2017-07-03 11:56:38.029,1499082998.0292608737945556640625


In [73]:
def generate_basic_features(dataset):
    colFlows=list()
    i=0
    new_df=pd.DataFrame(columns=dataset.columns)
    for index,data in dataset.groupby(['tcp_stream','protocol']):
        i+=1
        
        flow_list=list()
        # go through conversations (unidirectional flows) to generate the flows
        for index_conversation,conversation in data.iterrows():
            # check if conversation already exists
            isnewflow=True
            
            for f in flow_list:
                if (conversation.src_ip==f.get('src_ip',"") and conversation.src_port==f.get('src_port',"")) and (conversation.dst_ip==f.get('dst_ip',"") and conversation.dst_port==f.get('dst_port',"")):
                    isnewflow=False
                    f['fwd_packets']=f.get('fwd_packets',0)+conversation.total_packets
                    f['fwd_bytes']=f.get('fwd_bytes',0)+conversation.total_bytes
                    f['fwd_total_http_errors']=f.get('fwd_total_http_errors',0)+conversation.total_http_errors
                    f['fwd_total_failures']=f.get('fwd_total_failures',0)+conversation.total_failures
                elif (conversation.src_ip==f.get('dst_ip',"") and conversation.src_port==f.get('dst_port',"")) and (conversation.dst_ip==f.get('src_ip',"") and conversation.dst_port==f.get('src_port',"")):
                    isnewflow=False
                    f['bwd_packets']=f.get('bwd_packets',0)+conversation.total_packets
                    f['bwd_bytes']=f.get('bwd_bytes',0)+conversation.total_bytes
                    f['bwd_total_http_errors']=f.get('bwd_total_http_errors',0)+conversation.total_http_errors
                    f['bwd_total_failures']=f.get('bwd_total_failures',0)+conversation.total_failures
                    
            if isnewflow:
                new_flow={}
                
                new_flow['src_ip']=conversation.src_ip
                new_flow['dst_ip']=conversation.dst_ip
                new_flow['src_port']=conversation.src_port
                new_flow['dst_port']=conversation.dst_port
                new_flow['fwd_packets']=conversation.total_packets
                new_flow['bwd_packets']=0
                new_flow['fwd_bytes']=conversation.total_bytes
                new_flow['bwd_bytes']=0
                new_flow['fwd_total_http_errors']=conversation.total_http_errors
                new_flow['fwd_total_failures']=conversation.total_failures
                new_flow['tcp_stream']=index[0]
                new_flow['protocol']=index[1]
                flow_list.append(new_flow)
            
            
        
        
        colFlows.append(flow_list)
        
        
    return colFlows

In [74]:
cFlows=generate_basic_features(df[:20])

In [75]:
import itertools
merged = list(itertools.chain.from_iterable(cFlows))
df_merged=pd.DataFrame.from_dict(merged)

In [76]:
df_merged

,src_ip,dst_ip,src_port,dst_port,fwd_packets,bwd_packets,tcp_stream,protocol
0,192.168.10.14,13.107.4.50,50095,80,291,178,0,HTTP
1,192.168.10.15,13.107.4.50,49647,80,465,194,0,HTTP
2,23.194.140.85,192.168.10.16,443,49376,3,0,0,SSL
3,23.194.140.85,192.168.10.16,443,49376,1,0,0,SSLv2
4,8.254.250.126,192.168.10.5,80,49188,8,0,0,TCP
5,13.107.4.50,192.168.10.14,80,50095,718424,540924,0,TCP
6,13.107.4.50,192.168.10.14,80,51197,100976,80079,0,TCP
7,13.107.4.50,192.168.10.15,80,49647,1130439,849499,0,TCP
8,23.15.4.24,192.168.10.15,80,50098,142948,122687,0,TCP
9,192.168.10.16,23.194.140.85,49376,443,9,2,0,TCP


In [78]:
df[:30]

,tcp_stream,protocol,flow_id,flow_start,flow_finish,src_ip,src_port,dst_ip,dst_port,total_duration,...,bpp,total_http_errors,total_failures,flag_ack,flag_fin,flag_psh,flag_rst,flag_syn,first_flag,last_flag
2,0,HTTP,192.168.10.14:50095->13.107.4.50:80,2017-07-03 12:16:42.169,2017-07-03 12:21:57.939,192.168.10.14,50095,13.107.4.50,80,315.769901,...,366.707904,0,0,291,0,291,0,0,NaN,NaN
0,0,HTTP,13.107.4.50:80->192.168.10.14:50095,2017-07-03 12:16:44.006,2017-07-03 12:21:58.191,13.107.4.50,80,192.168.10.14,50095,314.185010,...,1914.471910,0,0,178,0,178,0,0,TCP Spurious Retransmission,TCP Spurious Retransmission
3,0,HTTP,192.168.10.15:49647->13.107.4.50:80,2017-07-03 12:34:53.358,2017-07-03 12:42:54.830,192.168.10.15,49647,13.107.4.50,80,481.471518,...,366.666667,0,0,465,0,465,0,0,NaN,NaN
1,0,HTTP,13.107.4.50:80->192.168.10.15:49647,2017-07-03 12:34:55.253,2017-07-03 12:42:53.495,13.107.4.50,80,192.168.10.15,49647,478.241280,...,2008.716495,0,0,194,0,194,0,0,TCP Spurious Retransmission,TCP Spurious Retransmission
4,0,SSL,23.194.140.85:443->192.168.10.16:49376,2017-07-03 13:18:37.636,2017-07-03 13:18:37.659,23.194.140.85,443,192.168.10.16,49376,0.023275,...,1099.666667,0,0,3,0,2,0,0,TCP Spurious Retransmission,TCP Spurious Retransmission
5,0,SSLv2,23.194.140.85:443->192.168.10.16:49376,2017-07-03 13:18:37.637,2017-07-03 13:18:37.637,23.194.140.85,443,192.168.10.16,49376,0.000000,...,1448.000000,0,0,1,0,0,0,0,NaN,NaN
18,0,TCP,8.254.250.126:80->192.168.10.5:49188,2017-07-03 11:55:58.598,2017-07-03 11:55:58.598,8.254.250.126,80,192.168.10.5,49188,0.000010,...,0.000000,0,0,8,8,0,0,0,"FIN, ACK",TCP Out-Of-Order
6,0,TCP,13.107.4.50:80->192.168.10.14:50095,2017-07-03 12:16:41.882,2017-07-03 12:23:07.935,13.107.4.50,80,192.168.10.14,50095,386.052819,...,2243.110026,0,0,718424,0,12360,1,0,ACK,"RST, ACK"
10,0,TCP,192.168.10.14:50095->13.107.4.50:80,2017-07-03 12:16:41.882,2017-07-03 12:21:58.241,192.168.10.14,50095,13.107.4.50,80,316.358993,...,0.000000,0,0,540924,0,0,0,0,ACK,ACK
7,0,TCP,13.107.4.50:80->192.168.10.14:51197,2017-07-03 12:28:24.847,2017-07-03 12:29:58.060,13.107.4.50,80,192.168.10.14,51197,93.212640,...,2823.852658,0,0,100976,0,2176,1,0,ACK,"RST, ACK"


In [40]:
pd.DataFrame([flow for flow in cFlows])

,0,1
0,"{'src_ip': '192.168.10.14', 'dst_ip': '13.107....","{'src_ip': '192.168.10.15', 'dst_ip': '13.107...."
1,"{'src_ip': '23.194.140.85', 'dst_ip': '192.168...",None


In [ ]:
import pickle